In [1]:
import mtt
import numpy as np
from copy import copy, deepcopy

In [2]:
# Default simulation values
obj1 = np.array([[0], [0], [1], [1]])
obj2 = np.array([[1], [1], [1], [1]])
initial = {0: obj1}
dt = 1
ep_normal = 1
ep_tangent = 1
nu = 1
ts = 6
miss_p = 0.01
lam = 0
fa_scale = 10
gen = mtt.MultiObjSimple(initial, dt, ep_tangent, ep_normal, nu)
params = gen.get_params()
P = params.pop("P")

In [3]:
params

{'f': <bound method MultiObjSimple.process_function of <mtt.multiobject_simple.MultiObjSimple object at 0x13f935c40>>,
 'A': <bound method MultiObjSimple.process_jacobian of <mtt.multiobject_simple.MultiObjSimple object at 0x13f935c40>>,
 'h': <bound method MultiObjSimple.measurement_function of <mtt.multiobject_simple.MultiObjSimple object at 0x13f935c40>>,
 'Q': array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]),
 'W': <bound method MultiObjSimple.W of <mtt.multiobject_simple.MultiObjSimple object at 0x13f935c40>>,
 'R': array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]),
 'H': array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])}

In [4]:
k = mtt.KalmanFilter(**params)

In [5]:
xt, P = k.time_update(obj1, P)
k.measurement_update(xt, P, obj2)

(array([[1.],
        [1.],
        [1.],
        [1.]]),
 array([[ 6.25000000e-01,  1.58539631e-19,  1.25000000e-01,
         -4.75618892e-19],
        [ 1.58539631e-19,  6.25000000e-01, -4.75618892e-19,
          1.25000000e-01],
        [ 1.25000000e-01, -4.75618892e-19,  6.25000000e-01,
          1.42685668e-18],
        [-4.75618892e-19,  1.25000000e-01,  1.42685668e-18,
          6.25000000e-01]]))

In [6]:
gate = mtt.DistanceGatingMHT(0.9)

tot = -1
tmm = -2
tnt = -3
main = mtt.TrackMaintenanceMHT(tot, tmm, tnt, 1-miss_p, 4, lam, params['R'], k)
hypo = mtt.HypothesisComp()
prune = mtt.Pruning(4)

tracker = mtt.MHTTracker(k, gate, main, hypo, prune)

In [7]:
tracker.measurements

[]

In [8]:
# test = np.zeros(4)
# test.shape = (4,1)
# tracker.predict([test])

In [9]:
tracker.tracks

[]

In [10]:
tracker.get_trajectories()

[]

In [11]:
tracker.tracks

[]

In [12]:
processes = dict()
measures = dict()
measure_colors = dict()
cur_seed = np.random.randint(10**7)
rng = np.random.default_rng(cur_seed)
process = gen.process(2, rng)
processes[len(processes.keys())] = process
measures[len(measures.keys())], measure_colors[len(measure_colors.keys())] = gen.measure(process, rng)
print(process[1])

{0: array([[1.        ],
       [1.        ],
       [0.06234308],
       [0.4236492 ]])}


In [13]:
for i in range(len(processes[0])):
    print(deepcopy(measures[0][i]))
    tracker.predict(deepcopy(measures[0][i]))

[array([[-0.80740756],
       [-0.97723542],
       [ 2.60972008],
       [ 3.49937978]])]
{0: 0}
[array([[ 2.07011995],
       [ 3.01176255],
       [-0.0033564 ],
       [-1.40967136]])]


/Users/eduardososa/projects/python/rips-2021-aerospace/mtt/mht/distances_mht.py:24: RuntimeWarning: invalid value encountered in sqrt
  dis = np.sqrt(innovation.T @ np.linalg.inv(K) @ innovation)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()